In [1]:
import numpy as np
import pandas as pd
import sqlite3

from algorithms.dcss import dcss
from algorithms.mds import mds
from algorithms.rank_k_svd import rank_k_svd
from algorithms.t_sne import t_sne

# Preprocessing

In [2]:
# Pull data from sqlite database
conn = sqlite3.connect('data/rpkm.db')
c = conn.cursor()
sql = '''
    SELECT *
    FROM rpkm
'''
c.execute(sql)
lines = c.fetchall()
c.execute('''PRAGMA table_info(rpkm);''')
cols = [line[1] for line in c.fetchall()]
conn.close()

In [3]:
df = pd.DataFrame(data=lines, columns=cols)
df.index=df['gene']
df = df.drop(['gene'], axis=1)

# sqlite doesn't handle large numbers of columns well, so we transpose in pandas
df = df.transpose()

# expand out the former column names to obtain a multi-index
df.index = df.index.str.split('_', expand=True)
df.index.names = ['organ', 'study', 'celltype', 'id']
df = df.sort_index()

In [4]:
# drop cells from very sparsely represented types
index_sizes = df.groupby(level=[0,1,2]).size()
print index_sizes
df = df.groupby(level=[0,1,2]).filter(lambda x: len(x) >= 5)

organ      study  celltype  
embryonic  deng   16cell        58
                  4cell         14
                  8cell         47
                  BXC           13
                  C57twocell     8
                  early2cell     8
                  earlyblast    43
                  fibroblast    10
                  late2cell     10
                  lateblast     30
                  mid2cell      12
                  midblast      60
                  zy1            1
                  zy2            1
                  zy3            1
                  zy4            1
dtype: int64
